In [1]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import NullFormatter
import datetime as dt
from math import sqrt, pow
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# warnings
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

%matplotlib inline

In [2]:
df_2021 = pd.read_csv("data/2021_Iowa_Liquor_Sales.csv", low_memory=False)

## Format data

In [3]:
df_2021['Date'] = pd.to_datetime(df_2021['Date'])

KeyError: 'Date'

In [ ]:
df_2021['Year'] = df_2021['Date'].dt.strftime('%Y')
df_2021['Month'] = df_2021['Date'].dt.strftime('%m')
df_2021['Day'] = df_2021['Date'].dt.strftime('%d')
df_2021['Day Of Week'] = df_2021['Date'].dt.day_name()

In [ ]:
df_2021['Month'] = df_2021['Month'].replace({'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'June', 
                                            '07':'Jul', '08': 'Aug','09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'})

In [ ]:
df_2021['Store Number'] = df_2021['Store Number'].astype('str')
df_2021['County Number'] = df_2021['County Number'].astype('str')
df_2021['Category'] = df_2021['Category'].astype('str')
df_2021['Item Number'] = df_2021['Item Number'].astype('str')
df_2021['Vendor Number'] = df_2021['Vendor Number'].astype('str')

In [ ]:
month_category = CategoricalDtype(categories=["Jan", "Feb", "Mar", "Apr", "May", "June", "Jul", "Aug",
                                            "Sep", "Oct", "Nov", "Dec"], ordered=True)

dayOfWeek_category = CategoricalDtype(categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", 
                                                  "Saturday", "Sunday"], ordered=True)

In [4]:
df_2021.head(5)

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,...,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-37974400176,2021-07-01,2572,Hy-Vee Food Store / Cedar Falls,6301 University,Cedar Falls,50613.0,POINT (-92.435236 42.512789),7.0,BLACK HAWK,...,86507,Paramount Triple Sec,12,1000,3.84,5.76,4,23.04,4.0,1.05
1,INV-33767900019,2021-01-25,4129,Cyclone Liquors,626 Lincoln Way,Ames,50010.0,POINT (-93.618911 42.022854),85.0,STORY,...,904563,SOOH Black Haus Blackberry Schnapps,12,1000,15.00,22.50,4,90.00,4.0,1.05
2,INV-33739700007,2021-01-25,6076,Shop N Save Newton,404 S 11th Ave W,Newton,50208.0,POINT (-93.058352 41.690355),50.0,JASPER,...,101187,Hennessy VS NBA 2020 Giftbox,12,750,20.99,31.49,60,1889.40,45.0,11.88
3,INV-37792000067,2021-06-24,5102,Wilkie Liquors,724 1st St SE,Mount Vernon,52314.0,POINT (-91.410401 41.918328),57.0,LINN,...,86251,Juarez Triple Sec,12,1000,2.42,3.63,60,217.80,60.0,15.85
4,INV-34644200110,2021-03-01,2647,Hy-Vee #7 / Cedar Rapids,5050 Edgewood Rd,Cedar Rapids,52411.0,POINT (-91.701581 42.030129),57.0,LINN,...,86843,Ole Smoky Salty Caramel Whiskey,6,750,10.00,15.00,18,270.00,13.5,3.56


In [ ]:
df_2021.info()

## Overall Sales of 2021

In [ ]:
sale_2021 = '${:,.0f}'.format(df_2021['Sale (Dollars)'].sum())
inv_2021 = '{:,.0f}'.format(len(df_2021['Invoice/Item Number']))

#-----------------------------------#
dow_sale_2021 = df_2021['Sale (Dollars)'].groupby(df_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_sale_2021 = dow_sale_2021.apply(lambda x: f'${x:,.0f}').to_frame()

dow_inv_2021 = df_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)
dow_inv_2021 = dow_inv_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

dow_2021 = pd.concat([dow_sale_2021, dow_inv_2021], axis=1, join="inner")

#-----------------------------------#
month_sale_2021 = df_2021['Sale (Dollars)'].groupby(df_2021['Month'].astype(month_category)).sum()
month_sale_2021 = month_sale_2021.apply(lambda x: f'${x:,.0f}').to_frame()

month_inv_2021 = df_2021['Month'].value_counts(sort=False)
month_inv_2021 = month_inv_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

month_bottle_2021 = df_2021['Bottles Sold'].groupby(df_2021['Month'].astype(month_category)).sum()
month_bottle_2021 = month_bottle_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

month_2021 = pd.concat([month_sale_2021, month_inv_2021, month_bottle_2021], axis=1, join="inner")

#-----------------------------------#
#store_sale_2021 = df_2021.groupby(['Store Number','Store Name','Address','City','Zip Code', 
#                                   'Store Location','County'])['Sale (Dollars)'].sum().nlargest(10)
store_sale_2021 = df_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
store_sale_2021 = store_sale_2021.apply(lambda x: f'${x:,.0f}').to_frame()

store_inv_2021 = df_2021['Store Name'].value_counts().nlargest(10)
store_inv_2021 = store_inv_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
city_sale_2021 = df_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   
city_sale_2021 = city_sale_2021.apply(lambda x: f'${x:,.0f}').to_frame()

city_inv_2021 = df_2021['City'].value_counts().nlargest(10)
city_inv_2021 = city_inv_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
county_sale_2021 = df_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   
county_sale_2021 = county_sale_2021.apply(lambda x: f'${x:,.0f}').to_frame()

county_inv_2021 = df_2021['County'].value_counts().nlargest(10)
county_inv_2021 = county_inv_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
catname_sale_2021 = df_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
catname_sale_2021 = catname_sale_2021.apply(lambda x: f'${x:,.0f}').to_frame()

catname_inv_2021 = df_2021['Category Name'].value_counts().nlargest(10)
catname_inv_2021 = catname_inv_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
vendor_sale_2021 = df_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
vendor_sale_2021 = vendor_sale_2021.apply(lambda x: f'${x:,.0f}').to_frame()

vendor_inv_2021 = df_2021['Vendor Name'].value_counts().nlargest(10)
vendor_inv_2021 = vendor_inv_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
item_sale_2021 = df_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   
item_sale_2021 = item_sale_2021.apply(lambda x: f'${x:,.0f}').to_frame()

item_inv_2021 = df_2021['Item Description'].value_counts().nlargest(10)
item_inv_2021 = item_inv_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
print(f"_____Overall sales of 2021_____\n")
print(f"Total sale of 2021: {sale_2021}")
print(f"Total invoice of 2021: {inv_2021}")
print("#-----------------------------------#")
print(f"Day of Week with Best Sales:\n {dow_2021}")
print("#-----------------------------------#")
print(f"Month with Best Sales:\n {month_2021}")
print("#-----------------------------------#")
print(f"Store (sales) with Best Sales:\n {store_sale_2021}")
print("#-----------------------------------#")
print(f"Store (invoices) with Best Sales:\n {store_inv_2021}")
print("#-----------------------------------#")
print(f"County (sales) with Best Sales:\n {county_sale_2021}")
print("#-----------------------------------#")
print(f"County (invoices) with Best Sales:\n {county_inv_2021}")
print("#-----------------------------------#")
print(f"City (sales) with Best Sales:\n {city_sale_2021}")
print("#-----------------------------------#")
print(f"City (invoices) with Best Sales:\n {city_inv_2021}")
print("#-----------------------------------#")
print(f"Category (sales) with Best Sales:\n {catname_sale_2021}")
print("#-----------------------------------#")
print(f"Category (invoices) with Best Sales:\n {catname_inv_2021}")
print("#-----------------------------------#")
print(f"Vendor (sales) with Best Sales:\n {vendor_sale_2021}")
print("#-----------------------------------#")
print(f"Vendor (invoices) with Best Sales:\n {vendor_inv_2021}")
print("#-----------------------------------#")
print(f"Item (sales) with Best Sales:\n {item_sale_2021}")
print("#-----------------------------------#")
print(f"Item (invoices) with Best Sales:\n {item_inv_2021}")

## Functions

In [ ]:
def reformat_large_tick_values(tick_val, pos):
    """
    Turns large tick values (in the billions, millions and thousands) such as 4500 into 4.5K and also appropriately turns 4000 into 4K (no zero after the decimal).
    """
    if tick_val >= 1000000000:
        val = round(tick_val/1000000000, 1)
        new_tick_format = '{:}B'.format(val)
    elif tick_val >= 1000000:
        val = round(tick_val/1000000, 1)
        new_tick_format = '{:}M'.format(val)
    elif tick_val >= 1000:
        val = round(tick_val/1000, 1)
        new_tick_format = '{:}K'.format(val)
    elif tick_val < 1000:
        new_tick_format = round(tick_val, 1)
    else:
        new_tick_format = tick_val

    # make new_tick_format into a string value
    new_tick_format = str(new_tick_format)
    
    # code below will keep 4.5M as is but change values such as 4.0M to 4M since that zero after the decimal isn't needed
    index_of_decimal = new_tick_format.find(".")
    
    if index_of_decimal != -1:
        value_after_decimal = new_tick_format[index_of_decimal+1]
        if value_after_decimal == "0":
            # remove the 0 after the decimal point since it's not needed
            new_tick_format = new_tick_format[0:index_of_decimal] + new_tick_format[index_of_decimal+2:]
            
    return new_tick_format

In [ ]:
# Day of Week with Best Sales
def plot_dow_bestsale_graph(df_sale, df_inv):     
    fig, ax1 = plt.subplots(figsize=(8, 5))

    ax2 = ax1.twinx()

    df_sale.plot(kind='bar', color='y', ax=ax1, label = 'Sales (USD)')
    df_inv.plot(kind='line', marker='d', ax=ax2, label='Number of invoices')
     
    ax1.set_ylabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    ax1.set_xlabel("Day of Week", fontsize=13, color='r', labelpad=15)
    ax1.set_xticklabels(['Mon', 'Tue','Wed','Thu','Fri','Sat','Sun'], rotation = 0)
    
    ax1.yaxis.tick_left()
    ax1.yaxis.set_major_formatter(reformat_large_tick_values)
    ax1.yaxis.set_minor_formatter(NullFormatter())

    ax2.set_ylabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    
    ax2.yaxis.tick_right()
    ax2.yaxis.set_major_formatter(reformat_large_tick_values)
    ax2.yaxis.set_minor_formatter(NullFormatter())

    # combine 2 legends into 1
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()

    lines = lines_1 + lines_2
    labels = labels_1 + labels_2

    ax1.legend(lines, labels, loc=0)
    ax2.legend().set_visible(False)

    plt.title("Day of Week with Best Sales", pad=20, fontsize=14); 

In [ ]:
# Month with Best Sales
def plot_month_bestsale_graph(df_sale, df_inv):     
    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax2 = ax1.twinx()

    df_sale.plot(kind='bar', color='y', ax=ax1, label='Sales (USD)')
    df_inv.plot(kind='line', marker='d', ax=ax2, label='Number of invoices')

       
    ax1.set_ylabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    ax1.set_xlabel("Months", fontsize=13, color='r', labelpad=15)
    ax1.set_xticklabels(['Jan', 'Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], rotation = 0)
    ax1.yaxis.tick_left()
    ax1.yaxis.set_major_formatter(reformat_large_tick_values)
    ax1.yaxis.set_minor_formatter(NullFormatter()) 

    ax2.set_ylabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    ax2.yaxis.tick_right()
    ax2.yaxis.set_major_formatter(reformat_large_tick_values)
    ax2.yaxis.set_minor_formatter(NullFormatter())

    # combine 2 legends into 1
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()

    lines = lines_1 + lines_2
    labels = labels_1 + labels_2

    ax1.legend(lines, labels, loc=0)
    ax2.legend().set_visible(False)

    plt.title("Months with Best Sales", pad=20, fontsize=14);

In [ ]:
def plot_top10store_bestsale_graph(df_store_sale):    
    ax = df_store_sale.plot.barh()

    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Store Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Sales in (USD)", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Stores with Best Sales (USD)", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10store_bestinv_graph(df_store_inv):    
    ax = df_store_inv.plot.barh()
    
    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Store Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Stores with Best number of invoices", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10city_bestsale_graph(df_city_sale):
    ax = df_city_sale.plot.barh()

    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("City Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 City with Best Sales (USD)", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10city_bestinv_graph(df_city_inv):    
    ax = df_city_inv.plot.barh()
    
    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("City Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 City with Best number of invoices", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10county_bestsale_graph(df_county_sale):
    ax = df_county_sale.plot.barh()

    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("County Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 County with Best Sales (USD)", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10county_bestinv_graph(df_county_inv):
    ax = df_county_inv.plot.barh()
    
    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("County Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 County with Best number of invoices", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10cat_bestsale_graph(df_cat_sale):
    ax = df_cat_sale.plot.barh()

    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Category Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Categories with Best Sales (USD)", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10cat_bestinv_graph(df_cat_inv):
    ax = df_cat_inv.plot.barh()
    
    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Categories Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Categories with Best number of invoices", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10vendors_bestsale_graph(df_vendor_sale):
    ax = df_vendor_sale.plot.barh()

    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Vendor Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Vendors with Best Sales (USD)", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10vendors_bestinv_graph(df_vendor_inv):
    ax = df_vendor_inv.plot.barh()
    
    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Vendor Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Vendors with Best number of invoices", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10items_bestsale_graph(df_item_sale):
    ax = df_item_sale.plot.barh()

    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Item Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Items with Best Sales (USD)", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10items_bestinv_graph(df_item_inv):
    ax = df_item_inv.plot.barh()
    
    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Item Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Items with Best number of invoices", fontsize=13, color='r', pad=15);

## Graphs

In [ ]:
dow_sale_2021 = df_2021['Sale (Dollars)'].groupby(df_2021['Day Of Week'].astype(dayOfWeek_category)).sum()
dow_inv_2021 = df_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)

plot_dow_bestsale_graph(dow_sale_2021, dow_inv_2021)

In [ ]:
month_sale_2021 = df_2021['Sale (Dollars)'].groupby(df_2021['Month'].astype(month_category)).sum()
month_inv_2021 = df_2021['Month'].value_counts(sort=False)

plot_month_bestsale_graph(month_sale_2021, month_inv_2021)

In [ ]:
store_sale_2021 = df_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10store_bestsale_graph(store_sale_2021)

In [ ]:
store_inv_2021 = df_2021['Store Name'].value_counts().nlargest(10)

plot_top10store_bestinv_graph(store_inv_2021)

In [ ]:
city_sale_2021 = df_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10city_bestsale_graph(city_sale_2021)

In [ ]:
city_inv_2021 = df_2021['City'].value_counts().nlargest(10)

plot_top10city_bestinv_graph(city_inv_2021)

In [ ]:
county_sale_2021 = df_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10county_bestsale_graph(county_sale_2021)

In [ ]:
county_inv_2021 = df_2021['County'].value_counts().nlargest(10)

plot_top10county_bestinv_graph(county_inv_2021)

In [ ]:
catname_sale_2021 = df_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10cat_bestsale_graph(catname_sale_2021)

In [ ]:
catname_inv_2021 = df_2021['Category Name'].value_counts().nlargest(10)

plot_top10cat_bestinv_graph(catname_inv_2021)

In [ ]:
vendor_sale_2021 = df_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10vendors_bestsale_graph(vendor_sale_2021)

In [ ]:
vendor_inv_2021 = df_2021['Vendor Name'].value_counts().nlargest(10)

plot_top10vendors_bestinv_graph(vendor_inv_2021)

In [ ]:
item_sale_2021 = df_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10items_bestsale_graph(item_sale_2021)

In [ ]:
item_inv_2021 = df_2021['Item Description'].value_counts().nlargest(10)

plot_top10items_bestinv_graph(item_inv_2021)

## Advanced Search

### Query By a Date

In [ ]:
# By a Date
# 2021-01-03

df_byDate_2021 = df_2021[df_2021['Date'] == '2021-01-05']

sale_byDate_2021 = '${:,.0f}'.format(df_byDate_2021['Sale (Dollars)'].sum())
inv_byDate_2021 = '{:,.0f}'.format(len(df_byDate_2021['Invoice/Item Number']))

#-----------------------------------#
dow_sale_byDate_2021 = df_byDate_2021['Sale (Dollars)'].groupby(df_byDate_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_sale_byDate_2021 = dow_sale_byDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

dow_inv_byDate_2021 = df_byDate_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)
dow_inv_byDate_2021 = dow_inv_byDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

dow_bottle_byDate_2021 = df_byDate_2021['Bottles Sold'].groupby(df_byDate_2021['Month'].astype(month_category)).sum()
dow_bottle_byDate_2021 = dow_bottle_byDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

dow_byDate_2021 = pd.concat([dow_sale_byDate_2021, dow_inv_byDate_2021], axis=1, join="inner")

#-----------------------------------#
month_sale_byDate_2021 = df_byDate_2021['Sale (Dollars)'].groupby(df_byDate_2021['Month'].astype(month_category)).sum()
month_sale_byDate_2021 = month_sale_byDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

month_inv_byDate_2021 = df_byDate_2021['Month'].value_counts(sort=False)
month_inv_byDate_2021 = month_inv_byDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

month_bottle_byDate_2021 = df_byDate_2021['Bottles Sold'].groupby(df_byDate_2021['Month'].astype(month_category)).sum()
month_bottle_byDate_2021 = month_bottle_byDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

month_byDate_2021 = pd.concat([month_sale_byDate_2021, month_inv_byDate_2021, month_bottle_byDate_2021], axis=1, join="inner")

#-----------------------------------#
#store_sale_2021 = df_2021.groupby(['Store Number','Store Name','Address','City','Zip Code', 
#                                   'Store Location','County'])['Sale (Dollars)'].sum().nlargest(10)
store_sale_byDate_2021 = df_byDate_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
store_sale_byDate_2021 = store_sale_byDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

store_inv_byDate_2021 = df_byDate_2021['Store Name'].value_counts().nlargest(10)
store_inv_byDate_2021 = store_inv_byDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
city_sale_byDate_2021 = df_byDate_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   
city_sale_byDate_2021 = city_sale_byDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

city_inv_byDate_2021 = df_byDate_2021['City'].value_counts().nlargest(10)
city_inv_byDate_2021 = city_inv_byDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
county_sale_byDate_2021 = df_byDate_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   
county_sale_byDate_2021 = county_sale_byDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

county_inv_byDate_2021 = df_byDate_2021['County'].value_counts().nlargest(10)
county_inv_byDate_2021 = county_inv_byDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
catname_sale_byDate_2021 = df_byDate_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
catname_sale_byDate_2021 = catname_sale_byDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

catname_inv_byDate_2021 = df_byDate_2021['Category Name'].value_counts().nlargest(10)
catname_inv_byDate_2021 = catname_inv_byDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
vendor_sale_byDate_2021 = df_byDate_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
vendor_sale_byDate_2021 = vendor_sale_byDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

vendor_inv_byDate_2021 = df_byDate_2021['Vendor Name'].value_counts().nlargest(10)
vendor_inv_byDate_2021 = vendor_inv_byDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
item_sale_byDate_2021 = df_byDate_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   
item_sale_byDate_2021 = item_sale_byDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

item_inv_byDate_2021 = df_byDate_2021['Item Description'].value_counts().nlargest(10)
item_inv_byDate_2021 = item_inv_byDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
print(f"_____2021-01-03 Sale_____\n")
print(f"Total sale: {sale_byDate_2021}")
print(f"Total invoice: {inv_byDate_2021}")
print("#-----------------------------------#")
print(f"Day of Week with Best Sales:\n {dow_byDate_2021}")
print("#-----------------------------------#")
print(f"Month with Best Sales:\n {month_byDate_2021}")
print("#-----------------------------------#")
print(f"Store (sales) with Best Sales:\n {store_sale_byDate_2021}")
print("#-----------------------------------#")
print(f"Store (invoices) with Best Sales:\n {store_inv_byDate_2021}")
print("#-----------------------------------#")
print(f"County (sales) with Best Sales:\n {city_sale_byDate_2021}")
print("#-----------------------------------#")
print(f"County (invoices) with Best Sales:\n {city_inv_byDate_2021}")
print("#-----------------------------------#")
print(f"City (sales) with Best Sales:\n {county_sale_byDate_2021}")
print("#-----------------------------------#")
print(f"City (invoices) with Best Sales:\n {county_inv_byDate_2021}")
print("#-----------------------------------#")
print(f"Category (sales) with Best Sales:\n {catname_sale_byDate_2021}")
print("#-----------------------------------#")
print(f"Category (invoices) with Best Sales:\n {catname_inv_byDate_2021}")
print("#-----------------------------------#")
print(f"Vendor (sales) with Best Sales:\n {vendor_sale_byDate_2021}")
print("#-----------------------------------#")
print(f"Vendor (invoices) with Best Sales:\n {vendor_inv_byDate_2021}")
print("#-----------------------------------#")
print(f"Item (sales) with Best Sales:\n {item_sale_byDate_2021}")
print("#-----------------------------------#")
print(f"Item (invoices) with Best Sales:\n {item_inv_byDate_2021}")

In [ ]:
# filter df with a date
def df_byDate(df, full_date):
    return df[df['Date'] == full_date]

In [ ]:
df_byDate_2021 = df_byDate(df_2021, '2021-01-02')

In [ ]:
dow_sale_byDate_2021 = df_byDate_2021['Sale (Dollars)'].groupby(df_byDate_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_inv_byDate_2021 = df_byDate_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)

plot_dow_bestsale_graph(dow_sale_byDate_2021, dow_inv_byDate_2021)

In [ ]:
month_sale_byDate_2021 = df_byDate_2021['Sale (Dollars)'].groupby(df_byDate_2021['Month'].astype(month_category)).sum()
month_inv_byDate_2021 = df_byDate_2021['Month'].value_counts(sort=False)

plot_month_bestsale_graph(month_sale_byDate_2021, month_inv_byDate_2021)

In [ ]:
store_sale_byDate_2021 = df_byDate_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)   

plot_top10store_bestsale_graph(store_sale_byDate_2021)

In [ ]:
store_inv_byDate_2021 = df_byDate_2021['Store Name'].value_counts().nlargest(10)

plot_top10store_bestinv_graph(store_inv_byDate_2021)

In [ ]:
city_sale_byDate_2021 = df_byDate_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)  

plot_top10city_bestsale_graph(city_sale_byDate_2021)

In [ ]:
city_inv_byDate_2021 = df_byDate_2021['City'].value_counts().nlargest(10)

plot_top10city_bestinv_graph(city_inv_byDate_2021)

In [ ]:
county_sale_byDate_2021 = df_byDate_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)   

plot_top10county_bestsale_graph(county_sale_byDate_2021)

In [ ]:
county_inv_byDate_2021 = df_byDate_2021['County'].value_counts().nlargest(10)

plot_top10county_bestinv_graph(county_inv_byDate_2021)

In [ ]:
catname_sale_byDate_2021 = df_byDate_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10cat_bestsale_graph(catname_sale_byDate_2021)

In [ ]:
catname_inv_byDate_2021 = df_byDate_2021['Category Name'].value_counts().nlargest(10)

plot_top10cat_bestinv_graph(catname_inv_byDate_2021)

In [ ]:
vendor_sale_byDate_2021 = df_byDate_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10vendors_bestsale_graph(vendor_sale_byDate_2021)

In [ ]:
vendor_inv_byDate_2021 = df_byDate_2021['Vendor Name'].value_counts().nlargest(10)

plot_top10vendors_bestinv_graph(vendor_inv_byDate_2021)

In [ ]:
item_sale_byDate_2021 = df_byDate_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10items_bestsale_graph(item_sale_byDate_2021)

In [ ]:
item_inv_byDate_2021 = df_byDate_2021['Item Description'].value_counts().nlargest(10)

plot_top10items_bestinv_graph(item_inv_byDate_2021)

### Query by Date to Date (Same year)

In [ ]:
# By Date
# 2021-01-03 to 2021-01-04

df_byDateToDate_2021 = df_2021[(df_2021['Date'] >= '2021-01-03') & (df_2021['Date'] <= '2021-01-04')]

sale_byDateToDate_2021 = '${:,.0f}'.format(df_byDateToDate_2021['Sale (Dollars)'].sum())
inv_byDateToDate_2021 = '{:,.0f}'.format(len(df_byDateToDate_2021['Invoice/Item Number']))

#-----------------------------------#
dow_sale_byDateToDate_2021 = df_byDateToDate_2021['Sale (Dollars)'].groupby(df_byDateToDate_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_sale_byDateToDate_2021 = dow_sale_byDateToDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

dow_inv_byDateToDate_2021 = df_byDateToDate_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)
dow_inv_byDateToDate_2021 = dow_inv_byDateToDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

dow_bottle_byDateToDate_2021 = df_byDateToDate_2021['Bottles Sold'].groupby(df_byDateToDate_2021['Month'].astype(month_category)).sum()
dow_bottle_byDateToDate_2021 = dow_bottle_byDateToDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

dow_byDateToDate_2021 = pd.concat([dow_sale_byDateToDate_2021, dow_inv_byDateToDate_2021], axis=1, join="inner")

#-----------------------------------#
month_sale_byDateToDate_2021 = df_byDateToDate_2021['Sale (Dollars)'].groupby(df_byDateToDate_2021['Month'].astype(month_category)).sum()
month_sale_byDateToDate_2021 = month_sale_byDateToDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

month_inv_byDateToDate_2021 = df_byDateToDate_2021['Month'].value_counts(sort=False)
month_inv_byDateToDate_2021 = month_inv_byDateToDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

month_bottle_byDateToDate_2021 = df_byDateToDate_2021['Bottles Sold'].groupby(df_byDateToDate_2021['Month'].astype(month_category)).sum()
month_bottle_byDateToDate_2021 = month_bottle_byDateToDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

month_byDateToDate_2021 = pd.concat([month_sale_byDateToDate_2021, month_inv_byDateToDate_2021, month_bottle_byDateToDate_2021], axis=1, join="inner")

#-----------------------------------#
#store_sale_2021 = df_2021.groupby(['Store Number','Store Name','Address','City','Zip Code', 
#                                   'Store Location','County'])['Sale (Dollars)'].sum().nlargest(10)
store_sale_byDateToDate_2021 = df_byDateToDate_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
store_sale_byDateToDate_2021 = store_sale_byDateToDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

store_inv_byDateToDate_2021 = df_byDateToDate_2021['Store Name'].value_counts().nlargest(10)
store_inv_byDateToDate_2021 = store_inv_byDateToDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
city_sale_byDateToDate_2021 = df_byDateToDate_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   
city_sale_byDateToDate_2021 = city_sale_byDateToDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

city_inv_byDateToDate_2021 = df_byDateToDate_2021['City'].value_counts().nlargest(10)
city_inv_byDateToDate_2021 = city_inv_byDateToDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
county_sale_byDateToDate_2021 = df_byDateToDate_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   
county_sale_byDateToDate_2021 = county_sale_byDateToDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

county_inv_byDateToDate_2021 = df_byDateToDate_2021['County'].value_counts().nlargest(10)
county_inv_byDateToDate_2021 = county_inv_byDateToDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
catname_sale_byDateToDate_2021 = df_byDateToDate_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
catname_sale_byDateToDate_2021 = catname_sale_byDateToDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

catname_inv_byDateToDate_2021 = df_byDateToDate_2021['Category Name'].value_counts().nlargest(10)
catname_inv_byDateToDate_2021 = catname_inv_byDateToDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
vendor_sale_byDateToDate_2021 = df_byDateToDate_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
vendor_sale_byDateToDate_2021 = vendor_sale_byDateToDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

vendor_inv_byDateToDate_2021 = df_byDateToDate_2021['Vendor Name'].value_counts().nlargest(10)
vendor_inv_byDateToDate_2021 = vendor_inv_byDateToDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
item_sale_byDateToDate_2021 = df_byDateToDate_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   
item_sale_byDateToDate_2021 = item_sale_byDateToDate_2021.apply(lambda x: f'${x:,.0f}').to_frame()

item_inv_byDateToDate_2021 = df_byDateToDate_2021['Item Description'].value_counts().nlargest(10)
item_inv_byDateToDate_2021 = item_inv_byDateToDate_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
print(f"_____2021-01-03 to 2021-01-04 Sale_____\n")
print(f"Total sale: {sale_byDateToDate_2021}")
print(f"Total invoice: {inv_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"Day of Week with Best Sales:\n {dow_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"Month with Best Sales:\n {month_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"Store (sales) with Best Sales:\n {store_sale_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"Store (invoices) with Best Sales:\n {store_inv_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"County (sales) with Best Sales:\n {city_sale_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"County (invoices) with Best Sales:\n {city_inv_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"City (sales) with Best Sales:\n {county_sale_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"City (invoices) with Best Sales:\n {county_inv_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"Category (sales) with Best Sales:\n {catname_sale_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"Category (invoices) with Best Sales:\n {catname_inv_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"Vendor (sales) with Best Sales:\n {vendor_sale_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"Vendor (invoices) with Best Sales:\n {vendor_inv_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"Item (sales) with Best Sales:\n {item_sale_byDateToDate_2021}")
print("#-----------------------------------#")
print(f"Item (invoices) with Best Sales:\n {item_inv_byDateToDate_2021}")

In [ ]:
# filter df with from date to date
def df_byDateToDate(df, fromDate, toDate):
    return df[(df['Date'] >= fromDate) & (df['Date'] <= toDate)]

In [ ]:
df_byDateToDate_2021 = df_byDateToDate(df_2021, '2021-01-02', '2021-01-05')

In [ ]:
dow_sale_byDateToDate_2021 = df_byDateToDate_2021['Sale (Dollars)'].groupby(df_byDateToDate_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_inv_byDateToDate_2021 = df_byDateToDate_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)

plot_dow_bestsale_graph(dow_sale_byDateToDate_2021, dow_inv_byDateToDate_2021)

In [ ]:
month_sale_byDateToDate_2021 = df_byDateToDate_2021['Sale (Dollars)'].groupby(df_byDateToDate_2021['Month'].astype(month_category)).sum()
month_inv_byDateToDate_2021 = df_byDateToDate_2021['Month'].value_counts(sort=False)

plot_month_bestsale_graph(month_sale_byDateToDate_2021, month_inv_byDateToDate_2021)

In [ ]:
store_sale_byDateToDate_2021 = df_byDateToDate_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10store_bestsale_graph(store_sale_byDateToDate_2021)

In [ ]:
store_inv_byDateToDate_2021 = df_byDateToDate_2021['Store Name'].value_counts().nlargest(10)

plot_top10store_bestinv_graph(store_inv_byDateToDate_2021)

In [ ]:
city_sale_byDateToDate_2021 = df_byDateToDate_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   


plot_top10city_bestsale_graph(city_sale_byDateToDate_2021)

In [ ]:
city_inv_byDateToDate_2021 = df_byDateToDate_2021['City'].value_counts().nlargest(10)

plot_top10city_bestinv_graph(city_inv_byDateToDate_2021)

In [ ]:
county_sale_byDateToDate_2021 = df_byDateToDate_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10county_bestsale_graph(county_sale_byDateToDate_2021)

In [ ]:
county_inv_byDateToDate_2021 = df_byDateToDate_2021['County'].value_counts().nlargest(10)

plot_top10county_bestinv_graph(county_inv_byDateToDate_2021)

In [ ]:
catname_sale_byDateToDate_2021 = df_byDateToDate_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10cat_bestsale_graph(catname_sale_byDateToDate_2021)

In [ ]:
catname_inv_byDateToDate_2021 = df_byDateToDate_2021['Category Name'].value_counts().nlargest(10)

plot_top10cat_bestinv_graph(catname_inv_byDateToDate_2021)

In [ ]:
vendor_sale_byDateToDate_2021 = df_byDateToDate_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10vendors_bestsale_graph(vendor_sale_byDateToDate_2021)

In [ ]:
vendor_inv_byDateToDate_2021 = df_byDateToDate_2021['Vendor Name'].value_counts().nlargest(10)

plot_top10vendors_bestinv_graph(vendor_inv_byDateToDate_2021)

In [ ]:
item_sale_byDateToDate_2021 = df_byDateToDate_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10items_bestsale_graph(item_sale_byDateToDate_2021)

In [ ]:
item_inv_byDateToDate_2021 = df_byDateToDate_2021['Item Description'].value_counts().nlargest(10)

plot_top10items_bestinv_graph(item_inv_byDateToDate_2021)

### Query By Store

In [ ]:
# By Store
# 2549

df_byStore_2021 = df_2021[df_2021['Store Number'] == '2549']

sale_byStore_2021 = '${:,.0f}'.format(df_byStore_2021['Sale (Dollars)'].sum())
inv_byStore_2021 = '{:,.0f}'.format(len(df_byStore_2021['Invoice/Item Number']))

#-----------------------------------#
dow_sale_byStore_2021 = df_byStore_2021['Sale (Dollars)'].groupby(df_byStore_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_sale_byStore_2021 = dow_sale_byStore_2021.apply(lambda x: f'${x:,.0f}').to_frame()

dow_inv_byStore_2021 = df_byStore_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)
dow_inv_byStore_2021 = dow_inv_byStore_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

dow_bottle_byStore_2021 = df_byStore_2021['Bottles Sold'].groupby(df_byStore_2021['Month'].astype(month_category)).sum()
dow_bottle_byStore_2021 = dow_bottle_byStore_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

dow_byStore_2021 = pd.concat([dow_sale_byStore_2021, dow_inv_byStore_2021], axis=1, join="inner")

#-----------------------------------#
month_sale_byStore_2021 = df_byStore_2021['Sale (Dollars)'].groupby(df_byStore_2021['Month'].astype(month_category)).sum()
month_sale_byStore_2021 = month_sale_byStore_2021.apply(lambda x: f'${x:,.0f}').to_frame()

month_inv_byStore_2021 = df_byStore_2021['Month'].value_counts(sort=False)
month_inv_byStore_2021 = month_inv_byStore_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

month_bottle_byStore_2021 = df_byStore_2021['Bottles Sold'].groupby(df_byStore_2021['Month'].astype(month_category)).sum()
month_bottle_byStore_2021 = month_bottle_byStore_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

month_byStore_2021 = pd.concat([month_sale_byStore_2021, month_inv_byStore_2021, month_bottle_byStore_2021], axis=1, join="inner")

#-----------------------------------#
catname_sale_byStore_2021 = df_byStore_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
catname_sale_byStore_2021 = catname_sale_byStore_2021.apply(lambda x: f'${x:,.0f}').to_frame()

catname_inv_byStore_2021 = df_byStore_2021['Category Name'].value_counts().nlargest(10)
catname_inv_byStore_2021 = catname_inv_byStore_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
vendor_sale_byStore_2021 = df_byStore_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
vendor_sale_byStore_2021 = vendor_sale_byStore_2021.apply(lambda x: f'${x:,.0f}').to_frame()

vendor_inv_byStore_2021 = df_byStore_2021['Vendor Name'].value_counts().nlargest(10)
vendor_inv_byStore_2021 = vendor_inv_byStore_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
item_sale_byStore_2021 = df_byStore_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   
item_sale_byStore_2021 = item_sale_byStore_2021.apply(lambda x: f'${x:,.0f}').to_frame()

item_inv_byStore_2021 = df_byStore_2021['Item Description'].value_counts().nlargest(10)
item_inv_byStore_2021 = item_inv_byStore_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
print(f"_____Store Sale_____\n")
print(f"Total sale: {sale_byStore_2021}")
print(f"Total invoice: {inv_byStore_2021}")
print("#-----------------------------------#")
print(f"Day of Week with Best Sales:\n {dow_byStore_2021}")
print("#-----------------------------------#")
print(f"Month with Best Sales:\n {month_byStore_2021}")
print("#-----------------------------------#")
print(f"Category (sales) with Best Sales:\n {catname_sale_byStore_2021}")
print("#-----------------------------------#")
print(f"Category (invoices) with Best Sales:\n {catname_inv_byStore_2021}")
print("#-----------------------------------#")
print(f"Vendor (sales) with Best Sales:\n {vendor_sale_byStore_2021}")
print("#-----------------------------------#")
print(f"Vendor (invoices) with Best Sales:\n {vendor_inv_byStore_2021}")
print("#-----------------------------------#")
print(f"Item (sales) with Best Sales:\n {item_sale_byStore_2021}")
print("#-----------------------------------#")
print(f"Item (invoices) with Best Sales:\n {item_inv_byStore_2021}")

In [ ]:
# filter df with from date to date
def df_byStore(df, storeName):
    return df[df['Store Number'] == storeName]

In [ ]:
df_byStore_2021 = df_byStore(df_2021, '2549')

In [ ]:
dow_sale_byStore_2021 = df_byStore_2021['Sale (Dollars)'].groupby(df_byStore_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_inv_byStore_2021 = df_byStore_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)

plot_dow_bestsale_graph(dow_sale_byStore_2021, dow_inv_byStore_2021)

In [ ]:
month_sale_byStore_2021 = df_byStore_2021['Sale (Dollars)'].groupby(df_byStore_2021['Month'].astype(month_category)).sum()
month_inv_byStore_2021 = df_byStore_2021['Month'].value_counts(sort=False)

plot_month_bestsale_graph(month_sale_byStore_2021, month_inv_byStore_2021)

In [ ]:
catname_sale_byStore_2021 = df_byStore_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10cat_bestsale_graph(catname_sale_byStore_2021)

In [ ]:
catname_inv_byStore_2021 = df_byStore_2021['Category Name'].value_counts().nlargest(10)

plot_top10cat_bestinv_graph(catname_inv_byStore_2021)

In [ ]:
vendor_sale_byStore_2021 = df_byStore_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10vendors_bestsale_graph(vendor_sale_byStore_2021)

In [ ]:
vendor_inv_byStore_2021 = df_byStore_2021['Vendor Name'].value_counts().nlargest(10)

plot_top10vendors_bestinv_graph(vendor_inv_byStore_2021)

In [ ]:
item_sale_byStore_2021 = df_byStore_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10items_bestsale_graph(item_sale_byStore_2021)

In [ ]:
item_inv_byStore_2021 = df_byStore_2021['Item Description'].value_counts().nlargest(10)

plot_top10items_bestinv_graph(item_inv_byStore_2021)

### Query by City

In [ ]:
# By City
# INDIANOLA

df_byCity_2021 = df_2021[df_2021['City'] == 'Ames']

sale_byCity_2021 = '${:,.0f}'.format(df_byCity_2021['Sale (Dollars)'].sum())
inv_byCity_2021 = '{:,.0f}'.format(len(df_byCity_2021['Invoice/Item Number']))

#-----------------------------------#
dow_sale_byCity_2021 = df_byCity_2021['Sale (Dollars)'].groupby(df_byCity_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_sale_byCity_2021 = dow_sale_byCity_2021.apply(lambda x: f'${x:,.0f}').to_frame()

dow_inv_byCity_2021 = df_byCity_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)
dow_inv_byCity_2021 = dow_inv_byCity_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

dow_bottle_byCity_2021 = df_byCity_2021['Bottles Sold'].groupby(df_byCity_2021['Month'].astype(month_category)).sum()
dow_bottle_byCity_2021 = dow_bottle_byCity_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

dow_byCity_2021 = pd.concat([dow_sale_byCity_2021, dow_inv_byCity_2021], axis=1, join="inner")

#-----------------------------------#
month_sale_byCity_2021 = df_byCity_2021['Sale (Dollars)'].groupby(df_byCity_2021['Month'].astype(month_category)).sum()
month_sale_byCity_2021 = month_sale_byCity_2021.apply(lambda x: f'${x:,.0f}').to_frame()

month_inv_byCity_2021 = df_byCity_2021['Month'].value_counts(sort=False)
month_inv_byCity_2021 = month_inv_byCity_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

month_bottle_byCity_2021 = df_byCity_2021['Bottles Sold'].groupby(df_byCity_2021['Month'].astype(month_category)).sum()
month_bottle_byCity_2021 = month_bottle_byCity_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

month_byCity_2021 = pd.concat([month_sale_byCity_2021, month_inv_byCity_2021, month_bottle_byCity_2021], axis=1, join="inner")

#-----------------------------------#
#store_sale_2021 = df_2021.groupby(['Store Number','Store Name','Address','City','Zip Code', 
#                                   'Store Location','County'])['Sale (Dollars)'].sum().nlargest(10)
store_sale_byCity_2021 = df_byCity_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
store_sale_byCity_2021 = store_sale_byCity_2021.apply(lambda x: f'${x:,.0f}').to_frame()

store_inv_byCity_2021 = df_byCity_2021['Store Name'].value_counts().nlargest(10)
store_inv_byCity_2021 = store_inv_byCity_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
catname_sale_byCity_2021 = df_byCity_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
catname_sale_byCity_2021 = catname_sale_byCity_2021.apply(lambda x: f'${x:,.0f}').to_frame()

catname_inv_byCity_2021 = df_byCity_2021['Category Name'].value_counts().nlargest(10)
catname_inv_byCity_2021 = catname_inv_byCity_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
vendor_sale_byCity_2021 = df_byCity_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
vendor_sale_byCity_2021 = vendor_sale_byCity_2021.apply(lambda x: f'${x:,.0f}').to_frame()

vendor_inv_byCity_2021 = df_byCity_2021['Vendor Name'].value_counts().nlargest(10)
vendor_inv_byCity_2021 = vendor_inv_byCity_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
item_sale_byCity_2021 = df_byCity_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   
item_sale_byCity_2021 = item_sale_byCity_2021.apply(lambda x: f'${x:,.0f}').to_frame()

item_inv_byCity_2021 = df_byCity_2021['Item Description'].value_counts().nlargest(10)
item_inv_byCity_2021 = item_inv_byCity_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
print(f"_____City Sale_____\n")
print(f"Total sale: {sale_byCity_2021}")
print(f"Total invoice: {inv_byCity_2021}")
print("#-----------------------------------#")
print(f"Day of Week with Best Sales:\n {dow_byCity_2021}")
print("#-----------------------------------#")
print(f"Month with Best Sales:\n {month_byCity_2021}")
print("#-----------------------------------#")
print(f"Store (sales) with Best Sales:\n {store_sale_byCity_2021}")
print("#-----------------------------------#")
print(f"Store (invoices) with Best Sales:\n {store_inv_byCity_2021}")
print("#-----------------------------------#")
print(f"Category (sales) with Best Sales:\n {catname_sale_byCity_2021}")
print("#-----------------------------------#")
print(f"Category (invoices) with Best Sales:\n {catname_inv_byCity_2021}")
print("#-----------------------------------#")
print(f"Vendor (sales) with Best Sales:\n {vendor_sale_byCity_2021}")
print("#-----------------------------------#")
print(f"Vendor (invoices) with Best Sales:\n {vendor_inv_byCity_2021}")
print("#-----------------------------------#")
print(f"Item (sales) with Best Sales:\n {item_sale_byCity_2021}")
print("#-----------------------------------#")
print(f"Item (invoices) with Best Sales:\n {item_inv_byCity_2021}")

In [ ]:
# filter df with from city
def df_byCity(df, cityName):
    return df[df['City'] == cityName]

In [ ]:
df_byCity_2021 = df_byCity(df_2021, 'Ames')

In [ ]:
dow_sale_byCity_2021 = df_byCity_2021['Sale (Dollars)'].groupby(df_byCity_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_inv_byCity_2021 = df_byCity_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)

plot_dow_bestsale_graph(dow_sale_byCity_2021, dow_inv_byCity_2021)

In [ ]:
month_sale_byCity_2021 = df_byCity_2021['Sale (Dollars)'].groupby(df_byCity_2021['Month'].astype(month_category)).sum()
month_inv_byCity_2021 = df_byCity_2021['Month'].value_counts(sort=False)

plot_month_bestsale_graph(month_sale_byCity_2021, month_inv_byCity_2021)

In [ ]:
store_sale_byCity_2021 = df_byCity_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10store_bestsale_graph(store_sale_byCity_2021)

In [ ]:
store_inv_byCity_2021 = df_byCity_2021['Store Name'].value_counts().nlargest(10)

plot_top10store_bestinv_graph(store_inv_byCity_2021)

In [ ]:
catname_sale_byCity_2021 = df_byCity_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10cat_bestsale_graph(catname_sale_byCity_2021)

In [ ]:
catname_inv_byCity_2021 = df_byCity_2021['Category Name'].value_counts().nlargest(10)

plot_top10cat_bestinv_graph(catname_inv_byCity_2021)

In [ ]:
vendor_sale_byCity_2021 = df_byCity_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10vendors_bestsale_graph(vendor_sale_byCity_2021)

In [ ]:
vendor_inv_byCity_2021 = df_byCity_2021['Vendor Name'].value_counts().nlargest(10)

plot_top10vendors_bestinv_graph(vendor_inv_byCity_2021)

In [ ]:
item_sale_byCity_2021 = df_byCity_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10items_bestsale_graph(item_sale_byCity_2021)

In [ ]:
item_inv_byCity_2021 = df_byCity_2021['Item Description'].value_counts().nlargest(10)

plot_top10items_bestinv_graph(item_inv_byCity_2021)

### By County

In [ ]:
# By County
# INDIANOLA

df_byCounty_2021 = df_2021[df_2021['County'] == 'POLK']

sale_byCounty_2021 = '${:,.0f}'.format(df_byCounty_2021['Sale (Dollars)'].sum())
inv_byCounty_2021 = '{:,.0f}'.format(len(df_byCounty_2021['Invoice/Item Number']))

#-----------------------------------#
dow_sale_byCounty_2021 = df_byCounty_2021['Sale (Dollars)'].groupby(df_byCounty_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_sale_byCounty_2021 = dow_sale_byCounty_2021.apply(lambda x: f'${x:,.0f}').to_frame()

dow_inv_byCounty_2021 = df_byCounty_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)
dow_inv_byCounty_2021 = dow_inv_byCounty_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

dow_bottle_byCounty_2021 = df_byCounty_2021['Bottles Sold'].groupby(df_byCounty_2021['Month'].astype(month_category)).sum()
dow_bottle_byCounty_2021 = dow_bottle_byCounty_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

dow_byCounty_2021 = pd.concat([dow_sale_byCounty_2021, dow_inv_byCounty_2021], axis=1, join="inner")

#-----------------------------------#
month_sale_byCounty_2021 = df_byCounty_2021['Sale (Dollars)'].groupby(df_byCounty_2021['Month'].astype(month_category)).sum()
month_sale_byCounty_2021 = month_sale_byCounty_2021.apply(lambda x: f'${x:,.0f}').to_frame()

month_inv_byCounty_2021 = df_byCounty_2021['Month'].value_counts(sort=False)
month_inv_byCounty_2021 = month_inv_byCounty_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

month_bottle_byCounty_2021 = df_byCounty_2021['Bottles Sold'].groupby(df_byCounty_2021['Month'].astype(month_category)).sum()
month_bottle_byCounty_2021 = month_bottle_byCounty_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

month_byCounty_2021 = pd.concat([month_sale_byCounty_2021, month_inv_byCounty_2021, month_bottle_byCounty_2021], axis=1, join="inner")

#-----------------------------------#
#store_sale_2021 = df_2021.groupby(['Store Number','Store Name','Address','City','Zip Code', 
#                                   'Store Location','County'])['Sale (Dollars)'].sum().nlargest(10)
store_sale_byCounty_2021 = df_byCounty_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
store_sale_byCounty_2021 = store_sale_byCounty_2021.apply(lambda x: f'${x:,.0f}').to_frame()

store_inv_byCounty_2021 = df_byCounty_2021['Store Name'].value_counts().nlargest(10)
store_inv_byCounty_2021 = store_inv_byCounty_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
city_sale_byCounty_2021 = df_byCounty_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   
city_sale_byCounty_2021 = city_sale_byCounty_2021.apply(lambda x: f'${x:,.0f}').to_frame()

city_inv_byCounty_2021 = df_byCounty_2021['City'].value_counts().nlargest(10)
city_inv_byCounty_2021 = city_inv_byCounty_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
catname_sale_byCounty_2021 = df_byCounty_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
catname_sale_byCounty_2021 = catname_sale_byCounty_2021.apply(lambda x: f'${x:,.0f}').to_frame()

catname_inv_byCounty_2021 = df_byCounty_2021['Category Name'].value_counts().nlargest(10)
catname_inv_byCounty_2021 = catname_inv_byCounty_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
vendor_sale_byCounty_2021 = df_byCounty_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
vendor_sale_byCounty_2021 = vendor_sale_byCounty_2021.apply(lambda x: f'${x:,.0f}').to_frame()

vendor_inv_byCounty_2021 = df_byCounty_2021['Vendor Name'].value_counts().nlargest(10)
vendor_inv_byCounty_2021 = vendor_inv_byCounty_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
item_sale_byCounty_2021 = df_byCounty_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   
item_sale_byCounty_2021 = item_sale_byCounty_2021.apply(lambda x: f'${x:,.0f}').to_frame()

item_inv_byCounty_2021 = df_byCounty_2021['Item Description'].value_counts().nlargest(10)
item_inv_byCounty_2021 = item_inv_byCounty_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
print(f"_____County Sale_____\n")
print(f"Total sale: {sale_byCounty_2021}")
print(f"Total invoice: {inv_byCounty_2021}")
print("#-----------------------------------#")
print(f"Day of Week with Best Sales:\n {dow_byCounty_2021}")
print("#-----------------------------------#")
print(f"Month with Best Sales:\n {month_byCounty_2021}")
print("#-----------------------------------#")
print(f"Store (sales) with Best Sales:\n {store_sale_byCounty_2021}")
print("#-----------------------------------#")
print(f"Store (invoices) with Best Sales:\n {store_inv_byCounty_2021}")
print("#-----------------------------------#")
print(f"City (sales) with Best Sales:\n {city_sale_byCounty_2021}")
print("#-----------------------------------#")
print(f"City (invoices) with Best Sales:\n {city_inv_byCounty_2021}")
print("#-----------------------------------#")
print(f"Category (sales) with Best Sales:\n {catname_sale_byCounty_2021}")
print("#-----------------------------------#")
print(f"Category (invoices) with Best Sales:\n {catname_inv_byCounty_2021}")
print("#-----------------------------------#")
print(f"Vendor (sales) with Best Sales:\n {vendor_sale_byCounty_2021}")
print("#-----------------------------------#")
print(f"Vendor (invoices) with Best Sales:\n {vendor_inv_byCounty_2021}")
print("#-----------------------------------#")
print(f"Item (sales) with Best Sales:\n {item_sale_byCounty_2021}")
print("#-----------------------------------#")
print(f"Item (invoices) with Best Sales:\n {item_inv_byCounty_2021}")

In [ ]:
# filter df with from county
def df_byCounty(df, countyName):
    return df[df['County'] == countyName]

In [ ]:
df_byCounty_2021 = df_byCounty(df_2021, 'POLK')

In [ ]:
dow_sale_byCounty_2021 = df_byCounty_2021['Sale (Dollars)'].groupby(df_byCounty_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_inv_byCounty_2021 = df_byCounty_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)

plot_dow_bestsale_graph(dow_sale_byCounty_2021, dow_inv_byCounty_2021)

In [ ]:
month_sale_byCounty_2021 = df_byCounty_2021['Sale (Dollars)'].groupby(df_byCounty_2021['Month'].astype(month_category)).sum()
month_inv_byCounty_2021 = df_byCounty_2021['Month'].value_counts(sort=False)

plot_month_bestsale_graph(month_sale_byCounty_2021, month_inv_byCounty_2021)

In [ ]:
store_sale_byCounty_2021 = df_byCounty_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10store_bestsale_graph(store_sale_byCounty_2021)

In [ ]:
store_inv_byCounty_2021 = df_byCounty_2021['Store Name'].value_counts().nlargest(10)

plot_top10store_bestinv_graph(store_inv_byCounty_2021)

In [ ]:
city_sale_byCounty_2021 = df_byCounty_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10city_bestsale_graph(city_sale_byCounty_2021)

In [ ]:
city_inv_byCounty_2021 = df_byCounty_2021['City'].value_counts().nlargest(10)

plot_top10city_bestinv_graph(city_inv_byCounty_2021)

In [ ]:
catname_sale_byCounty_2021 = df_byCounty_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10cat_bestsale_graph(catname_sale_byCounty_2021)

In [ ]:
catname_inv_byCounty_2021 = df_byCounty_2021['Category Name'].value_counts().nlargest(10)

plot_top10cat_bestinv_graph(catname_inv_byCounty_2021)

In [ ]:
vendor_sale_byCounty_2021 = df_byCounty_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10vendors_bestsale_graph(vendor_sale_byCounty_2021)

In [ ]:
vendor_inv_byCounty_2021 = df_byCounty_2021['Vendor Name'].value_counts().nlargest(10)

plot_top10vendors_bestinv_graph(vendor_inv_byCounty_2021)

In [ ]:
item_sale_byCounty_2021 = df_byCounty_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10items_bestsale_graph(item_sale_byCounty_2021)

In [ ]:
item_inv_byCounty_2021 = df_byCounty_2021['Item Description'].value_counts().nlargest(10)

plot_top10items_bestinv_graph(item_inv_byCounty_2021)

### Query by Category

In [ ]:
# By Category
# 1062300.0

df_byCat_2021 = df_2021[df_2021['Category'] == '1062300.0']

sale_byCat_2021 = '${:,.0f}'.format(df_byCat_2021['Sale (Dollars)'].sum())
inv_byCat_2021 = '{:,.0f}'.format(len(df_byCat_2021['Invoice/Item Number']))

#-----------------------------------#
dow_sale_byCat_2021 = df_byCat_2021['Sale (Dollars)'].groupby(df_byCat_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_sale_byCat_2021 = dow_sale_byCat_2021.apply(lambda x: f'${x:,.0f}').to_frame()

dow_inv_byCat_2021 = df_byCat_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)
dow_inv_byCat_2021 = dow_inv_byCat_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

dow_bottle_byCat_2021 = df_byCat_2021['Bottles Sold'].groupby(df_byCat_2021['Month'].astype(month_category)).sum()
dow_bottle_byCat_2021 = dow_bottle_byCat_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

dow_byCat_2021 = pd.concat([dow_sale_byCat_2021, dow_inv_byCat_2021], axis=1, join="inner")

#-----------------------------------#
month_sale_byCat_2021 = df_byCat_2021['Sale (Dollars)'].groupby(df_byCat_2021['Month'].astype(month_category)).sum()
month_sale_byCat_2021 = month_sale_byCat_2021.apply(lambda x: f'${x:,.0f}').to_frame()

month_inv_byCat_2021 = df_byCat_2021['Month'].value_counts(sort=False)
month_inv_byCat_2021 = month_inv_byCat_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

month_bottle_byCat_2021 = df_byCat_2021['Bottles Sold'].groupby(df_byCat_2021['Month'].astype(month_category)).sum()
month_bottle_byCat_2021 = month_bottle_byCat_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

month_byCat_2021 = pd.concat([month_sale_byCat_2021, month_inv_byCat_2021, month_bottle_byCat_2021], axis=1, join="inner")

#-----------------------------------#
#store_sale_2021 = df_2021.groupby(['Store Number','Store Name','Address','City','Zip Code', 
#                                   'Store Location','County'])['Sale (Dollars)'].sum().nlargest(10)
store_sale_byCat_2021 = df_byCat_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
store_sale_byCat_2021 = store_sale_byCat_2021.apply(lambda x: f'${x:,.0f}').to_frame()

store_inv_byCat_2021 = df_byCat_2021['Store Name'].value_counts().nlargest(10)
store_inv_byCat_2021 = store_inv_byCat_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
city_sale_byCat_2021 = df_byCat_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   
city_sale_byCat_2021 = city_sale_byCat_2021.apply(lambda x: f'${x:,.0f}').to_frame()

city_inv_byCat_2021 = df_byCat_2021['City'].value_counts().nlargest(10)
city_inv_byCat_2021 = city_inv_byCat_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
county_sale_byCat_2021 = df_byCat_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   
county_sale_byCat_2021 = county_sale_byCat_2021.apply(lambda x: f'${x:,.0f}').to_frame()

county_inv_byCat_2021 = df_byCat_2021['County'].value_counts().nlargest(10)
county_inv_byCat_2021 = county_inv_byCat_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
vendor_sale_byCat_2021 = df_byCat_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
vendor_sale_byCat_2021 = vendor_sale_byCat_2021.apply(lambda x: f'${x:,.0f}').to_frame()

vendor_inv_byCat_2021 = df_byCat_2021['Vendor Name'].value_counts().nlargest(10)
vendor_inv_byCat_2021 = vendor_inv_byCat_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
item_sale_byCat_2021 = df_byCat_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   
item_sale_byCat_2021 = item_sale_byCat_2021.apply(lambda x: f'${x:,.0f}').to_frame()

item_inv_byCat_2021 = df_byCat_2021['Item Description'].value_counts().nlargest(10)
item_inv_byCat_2021 = item_inv_byCat_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
print(f"_____Category Sale_____\n")
print(f"Total sale: {sale_byCat_2021}")
print(f"Total invoice: {inv_byCat_2021}")
print("#-----------------------------------#")
print(f"Day of Week with Best Sales:\n {dow_byCat_2021}")
print("#-----------------------------------#")
print(f"Month with Best Sales:\n {month_byCat_2021}")
print("#-----------------------------------#")
print(f"City (sales) with Best Sales:\n {city_sale_byCat_2021}")
print("#-----------------------------------#")
print(f"City (invoices) with Best Sales:\n {city_inv_byCat_2021}")
print("#-----------------------------------#")
print(f"County (sales) with Best Sales:\n {county_sale_byCat_2021}")
print("#-----------------------------------#")
print(f"County (invoices) with Best Sales:\n {county_inv_byCat_2021}")
print("#-----------------------------------#")
print(f"Vendor (sales) with Best Sales:\n {vendor_sale_byCat_2021}")
print("#-----------------------------------#")
print(f"Vendor (invoices) with Best Sales:\n {vendor_inv_byCat_2021}")
print("#-----------------------------------#")
print(f"Item (sales) with Best Sales:\n {item_sale_byCat_2021}")
print("#-----------------------------------#")
print(f"Item (invoices) with Best Sales:\n {item_inv_byCat_2021}")

In [ ]:
# filter df with from category
def df_byCat(df, catName):
    return df[df['Category'] == catName]

In [ ]:
df_byCat_2021 = df_byCat(df_2021, '1062300.0')

In [ ]:
dow_sale_byCat_2021 = df_byCat_2021['Sale (Dollars)'].groupby(df_byCat_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_inv_byCat_2021 = df_byCat_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)

plot_dow_bestsale_graph(dow_sale_byCat_2021, dow_inv_byCat_2021)

In [ ]:
month_sale_byCat_2021 = df_byCat_2021['Sale (Dollars)'].groupby(df_byCat_2021['Month'].astype(month_category)).sum()
month_inv_byCat_2021 = df_byCat_2021['Month'].value_counts(sort=False)

plot_month_bestsale_graph(month_sale_byCat_2021, month_inv_byCat_2021)

In [ ]:
store_sale_byCat_2021 = df_byCat_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10store_bestsale_graph(store_sale_byCat_2021)

In [ ]:
store_inv_byCat_2021 = df_byCat_2021['Store Name'].value_counts().nlargest(10)

plot_top10store_bestinv_graph(store_inv_byCat_2021)

In [ ]:
city_sale_byCat_2021 = df_byCat_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10city_bestsale_graph(city_sale_byCat_2021)

In [ ]:
city_inv_byCat_2021 = df_byCat_2021['City'].value_counts().nlargest(10)

plot_top10city_bestinv_graph(city_inv_byCat_2021)

In [ ]:
county_sale_byCat_2021 = df_byCat_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10county_bestsale_graph(county_sale_byCat_2021)

In [ ]:
county_inv_byCat_2021 = df_byCat_2021['County'].value_counts().nlargest(10)

plot_top10county_bestinv_graph(county_inv_byCat_2021)

In [ ]:
vendor_sale_byCat_2021 = df_byCat_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10vendors_bestsale_graph(vendor_sale_byCat_2021)

In [ ]:
vendor_inv_byCat_2021 = df_byCat_2021['Vendor Name'].value_counts().nlargest(10)

plot_top10vendors_bestinv_graph(vendor_inv_byCat_2021)

In [ ]:
item_sale_byCat_2021 = df_byCat_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10items_bestsale_graph(item_sale_byCat_2021)

In [ ]:
item_inv_byCat_2021 = df_byCat_2021['Item Description'].value_counts().nlargest(10)

plot_top10items_bestinv_graph(item_inv_byCat_2021)

### Query by Vendor

In [ ]:
# By Vendor
# 370

df_byVendor_2021 = df_2021[df_2021['Vendor Number'] == '370']

sale_byVendor_2021 = '${:,.0f}'.format(df_byVendor_2021['Sale (Dollars)'].sum())
inv_byVendor_2021 = '{:,.0f}'.format(len(df_byVendor_2021['Invoice/Item Number']))

#-----------------------------------#
dow_sale_byVendor_2021 = df_byVendor_2021['Sale (Dollars)'].groupby(df_byVendor_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_sale_byVendor_2021 = dow_sale_byVendor_2021.apply(lambda x: f'${x:,.0f}').to_frame()

dow_inv_byVendor_2021 = df_byVendor_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)
dow_inv_byVendor_2021 = dow_inv_byVendor_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

dow_bottle_byVendor_2021 = df_byVendor_2021['Bottles Sold'].groupby(df_byVendor_2021['Month'].astype(month_category)).sum()
dow_bottle_byVendor_2021 = dow_bottle_byVendor_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

dow_byVendor_2021 = pd.concat([dow_sale_byVendor_2021, dow_inv_byVendor_2021], axis=1, join="inner")

#-----------------------------------#
month_sale_byVendor_2021 = df_byVendor_2021['Sale (Dollars)'].groupby(df_byVendor_2021['Month'].astype(month_category)).sum()
month_sale_byVendor_2021 = month_sale_byVendor_2021.apply(lambda x: f'${x:,.0f}').to_frame()

month_inv_byVendor_2021 = df_byVendor_2021['Month'].value_counts(sort=False)
month_inv_byVendor_2021 = month_inv_byVendor_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

month_bottle_byVendor_2021 = df_byVendor_2021['Bottles Sold'].groupby(df_byVendor_2021['Month'].astype(month_category)).sum()
month_bottle_byVendor_2021 = month_bottle_byVendor_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

month_byVendor_2021 = pd.concat([month_sale_byVendor_2021, month_inv_byVendor_2021, month_bottle_byVendor_2021], axis=1, join="inner")

#-----------------------------------#
#store_sale_2021 = df_2021.groupby(['Store Number','Store Name','Address','City','Zip Code', 
#                                   'Store Location','County'])['Sale (Dollars)'].sum().nlargest(10)
store_sale_byVendor_2021 = df_byVendor_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
store_sale_byVendor_2021 = store_sale_byVendor_2021.apply(lambda x: f'${x:,.0f}').to_frame()

store_inv_byVendor_2021 = df_byVendor_2021['Store Name'].value_counts().nlargest(10)
store_inv_byVendor_2021 = store_inv_byVendor_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
city_sale_byVendor_2021 = df_byVendor_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   
city_sale_byVendor_2021 = city_sale_byVendor_2021.apply(lambda x: f'${x:,.0f}').to_frame()

city_inv_byVendor_2021 = df_byVendor_2021['City'].value_counts().nlargest(10)
city_inv_byVendor_2021 = city_inv_byVendor_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
county_sale_byVendor_2021 = df_byVendor_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   
county_sale_byVendor_2021 = county_sale_byVendor_2021.apply(lambda x: f'${x:,.0f}').to_frame()

county_inv_byVendor_2021 = df_byVendor_2021['County'].value_counts().nlargest(10)
county_inv_byVendor_2021 = county_inv_byVendor_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
catname_sale_byVendor_2021 = df_byVendor_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
catname_sale_byVendor_2021 = catname_sale_byVendor_2021.apply(lambda x: f'${x:,.0f}').to_frame()

catname_inv_byVendor_2021 = df_byVendor_2021['Category Name'].value_counts().nlargest(10)
catname_inv_byVendor_2021 = catname_inv_byVendor_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
item_sale_byVendor_2021 = df_byVendor_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   
item_sale_byVendor_2021 = item_sale_byVendor_2021.apply(lambda x: f'${x:,.0f}').to_frame()

item_inv_byVendor_2021 = df_byVendor_2021['Item Description'].value_counts().nlargest(10)
item_inv_byVendor_2021 = item_inv_byVendor_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
print(f"_____Vendor Sale_____\n")
print(f"Total sale: {sale_byVendor_2021}")
print(f"Total invoice: {inv_byVendor_2021}")
print("#-----------------------------------#")
print(f"Day of Week with Best Sales:\n {dow_byVendor_2021}")
print("#-----------------------------------#")
print(f"Month with Best Sales:\n {month_byVendor_2021}")
print("#-----------------------------------#")
print(f"Store (sales) with Best Sales:\n {store_sale_byVendor_2021}")
print("#-----------------------------------#")
print(f"Store (invoices) with Best Sales:\n {store_inv_byVendor_2021}")
print("#-----------------------------------#")
print(f"County (sales) with Best Sales:\n {city_sale_byVendor_2021}")
print("#-----------------------------------#")
print(f"County (invoices) with Best Sales:\n {city_inv_byVendor_2021}")
print("#-----------------------------------#")
print(f"City (sales) with Best Sales:\n {county_sale_byVendor_2021}")
print("#-----------------------------------#")
print(f"City (invoices) with Best Sales:\n {county_inv_byVendor_2021}")
print("#-----------------------------------#")
print(f"Category (sales) with Best Sales:\n {catname_sale_byVendor_2021}")
print("#-----------------------------------#")
print(f"Category (invoices) with Best Sales:\n {catname_inv_byVendor_2021}")
print("#-----------------------------------#")
print(f"Item (sales) with Best Sales:\n {item_sale_byVendor_2021}")
print("#-----------------------------------#")
print(f"Item (invoices) with Best Sales:\n {item_inv_byVendor_2021}")

In [ ]:
# filter df with from Vendor name
def df_byVendor(df, vendorName):
    return df[df['Vendor Name'] == vendorName]

In [ ]:
df_byVendor_2021 = df_byVendor(df_2021, 'JIM BEAM BRANDS')

In [ ]:
dow_sale_byVendor_2021 = df_byVendor_2021['Sale (Dollars)'].groupby(df_byVendor_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_inv_byVendor_2021 = df_byVendor_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)

plot_dow_bestsale_graph(dow_sale_byVendor_2021, dow_inv_byVendor_2021)

In [ ]:
month_sale_byVendor_2021 = df_byVendor_2021['Sale (Dollars)'].groupby(df_byVendor_2021['Month'].astype(month_category)).sum()
month_inv_byVendor_2021 = df_byVendor_2021['Month'].value_counts(sort=False)

plot_month_bestsale_graph(month_sale_byVendor_2021, month_inv_byVendor_2021)

In [ ]:
store_sale_byVendor_2021 = df_byVendor_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10store_bestsale_graph(store_sale_byVendor_2021)

In [ ]:
store_inv_byVendor_2021 = df_byVendor_2021['Store Name'].value_counts().nlargest(10)

plot_top10store_bestinv_graph(store_inv_byVendor_2021)

In [ ]:
city_sale_byVendor_2021 = df_byVendor_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10city_bestsale_graph(city_sale_byVendor_2021)

In [ ]:
city_inv_byVendor_2021 = df_byVendor_2021['City'].value_counts().nlargest(10)

plot_top10city_bestinv_graph(city_inv_byVendor_2021)

In [ ]:
county_sale_byVendor_2021 = df_byVendor_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10county_bestsale_graph(county_sale_byVendor_2021)

In [ ]:
county_inv_byVendor_2021 = df_byVendor_2021['County'].value_counts().nlargest(10)

plot_top10county_bestinv_graph(county_inv_byVendor_2021)

In [ ]:
catname_sale_byVendor_2021 = df_byVendor_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10cat_bestsale_graph(catname_sale_byVendor_2021)

In [ ]:
catname_inv_byVendor_2021 = df_byVendor_2021['Category Name'].value_counts().nlargest(10)

plot_top10cat_bestinv_graph(catname_inv_byVendor_2021)

In [ ]:
item_sale_byVendor_2021 = df_byVendor_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10items_bestsale_graph(item_sale_byVendor_2021)

In [ ]:
item_inv_byVendor_2021 = df_byVendor_2021['Item Description'].value_counts().nlargest(10)

plot_top10items_bestinv_graph(item_inv_byVendor_2021)

### Query by Item

In [ ]:
# By Item
# 42717

df_byItem_2021 = df_2021[(df_2021['Item Number'] == '42717')]

sale_byItem_2021 = '${:,.0f}'.format(df_byItem_2021['Sale (Dollars)'].sum())
inv_byItem_2021 = '{:,.0f}'.format(len(df_byItem_2021['Invoice/Item Number']))

#-----------------------------------#
dow_sale_byItem_2021 = df_byItem_2021['Sale (Dollars)'].groupby(df_byItem_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_sale_byItem_2021 = dow_sale_byItem_2021.apply(lambda x: f'${x:,.0f}').to_frame()

dow_inv_byItem_2021 = df_byItem_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)
dow_inv_byItem_2021 = dow_inv_byItem_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

dow_bottle_byItem_2021 = df_byItem_2021['Bottles Sold'].groupby(df_byItem_2021['Month'].astype(month_category)).sum()
dow_bottle_byItem_2021 = dow_bottle_byItem_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

dow_byItem_2021 = pd.concat([dow_sale_byItem_2021, dow_inv_byItem_2021], axis=1, join="inner")

#-----------------------------------#
month_sale_byItem_2021 = df_byItem_2021['Sale (Dollars)'].groupby(df_byItem_2021['Month'].astype(month_category)).sum()
month_sale_byItem_2021 = month_sale_byItem_2021.apply(lambda x: f'${x:,.0f}').to_frame()

month_inv_byItem_2021 = df_byItem_2021['Month'].value_counts(sort=False)
month_inv_byItem_2021 = month_inv_byItem_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

month_bottle_byItem_2021 = df_byItem_2021['Bottles Sold'].groupby(df_byItem_2021['Month'].astype(month_category)).sum()
month_bottle_byItem_2021 = month_bottle_byItem_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

month_byItem_2021 = pd.concat([month_sale_byItem_2021, month_inv_byItem_2021, month_bottle_byItem_2021], axis=1, join="inner")

#-----------------------------------#
#store_sale_2021 = df_2021.groupby(['Store Number','Store Name','Address','City','Zip Code', 
#                                   'Store Location','County'])['Sale (Dollars)'].sum().nlargest(10)
store_sale_byItem_2021 = df_byItem_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
store_sale_byItem_2021 = store_sale_byItem_2021.apply(lambda x: f'${x:,.0f}').to_frame()

store_inv_byItem_2021 = df_byItem_2021['Store Name'].value_counts().nlargest(10)
store_inv_byItem_2021 = store_inv_byItem_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
city_sale_byItem_2021 = df_byItem_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   
city_sale_byItem_2021 = city_sale_byItem_2021.apply(lambda x: f'${x:,.0f}').to_frame()

city_inv_byItem_2021 = df_byItem_2021['City'].value_counts().nlargest(10)
city_inv_byItem_2021 = city_inv_byItem_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
county_sale_byItem_2021 = df_byItem_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   
county_sale_byItem_2021 = county_sale_byItem_2021.apply(lambda x: f'${x:,.0f}').to_frame()

county_inv_byItem_2021 = df_byItem_2021['County'].value_counts().nlargest(10)
county_inv_byItem_2021 = county_inv_byItem_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
print(f"_____Item Sale_____\n")
print(f"Total sale: {sale_byItem_2021}")
print(f"Total invoice: {inv_byItem_2021}")
print("#-----------------------------------#")
print(f"Day of Week with Best Sales:\n {dow_byItem_2021}")
print("#-----------------------------------#")
print(f"Month with Best Sales:\n {month_byItem_2021}")
print("#-----------------------------------#")
print(f"Store (sales) with Best Sales:\n {store_sale_byItem_2021}")
print("#-----------------------------------#")
print(f"Store (invoices) with Best Sales:\n {store_inv_byItem_2021}")
print("#-----------------------------------#")
print(f"County (sales) with Best Sales:\n {city_sale_byItem_2021}")
print("#-----------------------------------#")
print(f"County (invoices) with Best Sales:\n {city_inv_byItem_2021}")
print("#-----------------------------------#")
print(f"City (sales) with Best Sales:\n {county_sale_byItem_2021}")
print("#-----------------------------------#")
print(f"City (invoices) with Best Sales:\n {county_inv_byItem_2021}")

In [ ]:
# filter df with from Item name
def df_byItem(df, itemName):
    return df[df['Item Number'] == itemName]

In [ ]:
df_byItem_2021 = df_byItem(df_2021, '42717')

In [ ]:
dow_sale_byItem_2021 = df_byItem_2021['Sale (Dollars)'].groupby(df_byItem_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_inv_byItem_2021 = df_byItem_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)

plot_dow_bestsale_graph(dow_sale_byItem_2021, dow_inv_byItem_2021)

In [ ]:
month_sale_byItem_2021 = df_byItem_2021['Sale (Dollars)'].groupby(df_byItem_2021['Month'].astype(month_category)).sum()
month_inv_byItem_2021 = df_byItem_2021['Month'].value_counts(sort=False)

plot_month_bestsale_graph(month_sale_byItem_2021, month_inv_byItem_2021)

In [ ]:
store_sale_byItem_2021 = df_byItem_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10store_bestsale_graph(store_sale_byItem_2021)

In [ ]:
store_inv_byItem_2021 = df_byItem_2021['Store Name'].value_counts().nlargest(10)

plot_top10store_bestinv_graph(store_inv_byItem_2021)

In [ ]:
city_sale_byItem_2021 = df_byItem_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10city_bestsale_graph(city_sale_byItem_2021)

In [ ]:
city_inv_byItem_2021 = df_byItem_2021['City'].value_counts().nlargest(10)

plot_top10city_bestinv_graph(city_inv_byItem_2021)

In [ ]:
county_sale_byItem_2021 = df_byItem_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   

plot_top10county_bestsale_graph(county_sale_byItem_2021)

In [ ]:
county_inv_byItem_2021 = df_byItem_2021['County'].value_counts().nlargest(10)

plot_top10county_bestinv_graph(county_inv_byItem_2021)

### Query by Month to Month

In [ ]:
# By Month
# 2021-01 to 2021-02

df_byMonthToMonth_2021 = df_2021[(df_2021['Month'] == 'Jan')]

sale_byMonthToMonth_2021 = '${:,.0f}'.format(df_byMonthToMonth_2021['Sale (Dollars)'].sum())
inv_byMonthToMonth_2021 = '{:,.0f}'.format(len(df_byMonthToMonth_2021['Invoice/Item Number']))

#-----------------------------------#
dow_sale_byMonthToMonth_2021 = df_byMonthToMonth_2021['Sale (Dollars)'].groupby(df_byMonthToMonth_2021['Day Of Week'].astype(dayOfWeek_category)).sum()                         
dow_sale_byMonthToMonth_2021 = dow_sale_byMonthToMonth_2021.apply(lambda x: f'${x:,.0f}').to_frame()

dow_inv_byMonthToMonth_2021 = df_byMonthToMonth_2021['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)
dow_inv_byMonthToMonth_2021 = dow_inv_byMonthToMonth_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

dow_bottle_byMonthToMonth_2021 = df_byMonthToMonth_2021['Bottles Sold'].groupby(df_byMonthToMonth_2021['Month'].astype(month_category)).sum()
dow_bottle_byMonthToMonth_2021 = dow_bottle_byMonthToMonth_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

dow_byMonthToMonth_2021 = pd.concat([dow_sale_byMonthToMonth_2021, dow_inv_byMonthToMonth_2021], axis=1, join="inner")

#-----------------------------------#
month_sale_byMonthToMonth_2021 = df_byMonthToMonth_2021['Sale (Dollars)'].groupby(df_byMonthToMonth_2021['Month'].astype(month_category)).sum()
month_sale_byMonthToMonth_2021 = month_sale_byMonthToMonth_2021.apply(lambda x: f'${x:,.0f}').to_frame()

month_inv_byMonthToMonth_2021 = df_byMonthToMonth_2021['Month'].value_counts(sort=False)
month_inv_byMonthToMonth_2021 = month_inv_byMonthToMonth_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

month_bottle_byMonthToMonth_2021 = df_byMonthToMonth_2021['Bottles Sold'].groupby(df_byMonthToMonth_2021['Month'].astype(month_category)).sum()
month_bottle_byMonthToMonth_2021 = month_bottle_byMonthToMonth_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

month_byMonthToMonth_2021 = pd.concat([month_sale_byMonthToMonth_2021, month_inv_byMonthToMonth_2021, month_bottle_byMonthToMonth_2021], axis=1, join="inner")

#-----------------------------------#
#store_sale_2021 = df_2021.groupby(['Store Number','Store Name','Address','City','Zip Code', 
#                                   'Store Location','County'])['Sale (Dollars)'].sum().nlargest(10)
store_sale_byMonthToMonth_2021 = df_byMonthToMonth_2021.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
store_sale_byMonthToMonth_2021 = store_sale_byMonthToMonth_2021.apply(lambda x: f'${x:,.0f}').to_frame()

store_inv_byMonthToMonth_2021 = df_byMonthToMonth_2021['Store Name'].value_counts().nlargest(10)
store_inv_byMonthToMonth_2021 = store_inv_byMonthToMonth_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
city_sale_byMonthToMonth_2021 = df_byMonthToMonth_2021.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   
city_sale_byMonthToMonth_2021 = city_sale_byMonthToMonth_2021.apply(lambda x: f'${x:,.0f}').to_frame()

city_inv_byMonthToMonth_2021 = df_byMonthToMonth_2021['City'].value_counts().nlargest(10)
city_inv_byMonthToMonth_2021 = city_inv_byMonthToMonth_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
county_sale_byMonthToMonth_2021 = df_byMonthToMonth_2021.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   
county_sale_byMonthToMonth_2021 = county_sale_byMonthToMonth_2021.apply(lambda x: f'${x:,.0f}').to_frame()

county_inv_byMonthToMonth_2021 = df_byMonthToMonth_2021['County'].value_counts().nlargest(10)
county_inv_byMonthToMonth_2021 = county_inv_byMonthToMonth_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
catname_sale_byMonthToMonth_2021 = df_byMonthToMonth_2021.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
catname_sale_byMonthToMonth_2021 = catname_sale_byMonthToMonth_2021.apply(lambda x: f'${x:,.0f}').to_frame()

catname_inv_byMonthToMonth_2021 = df_byMonthToMonth_2021['Category Name'].value_counts().nlargest(10)
catname_inv_byMonthToMonth_2021 = catname_inv_byMonthToMonth_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
vendor_sale_byMonthToMonth_2021 = df_byMonthToMonth_2021.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
vendor_sale_byMonthToMonth_2021 = vendor_sale_byMonthToMonth_2021.apply(lambda x: f'${x:,.0f}').to_frame()

vendor_inv_byMonthToMonth_2021 = df_byMonthToMonth_2021['Vendor Name'].value_counts().nlargest(10)
vendor_inv_byMonthToMonth_2021 = vendor_inv_byMonthToMonth_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
item_sale_byMonthToMonth_2021 = df_byMonthToMonth_2021.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   
item_sale_byMonthToMonth_2021 = item_sale_byMonthToMonth_2021.apply(lambda x: f'${x:,.0f}').to_frame()

item_inv_byMonthToMonth_2021 = df_byMonthToMonth_2021['Item Description'].value_counts().nlargest(10)
item_inv_byMonthToMonth_2021 = item_inv_byMonthToMonth_2021.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoice')

#-----------------------------------#
print(f"_____2021-01 to 2021-02 Sale_____\n")
print(f"Total sale: {sale_byMonthToMonth_2021}")
print(f"Total invoice: {inv_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"Day of Week with Best Sales:\n {dow_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"Month with Best Sales:\n {month_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"Store (sales) with Best Sales:\n {store_sale_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"Store (invoices) with Best Sales:\n {store_inv_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"County (sales) with Best Sales:\n {city_sale_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"County (invoices) with Best Sales:\n {city_inv_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"City (sales) with Best Sales:\n {county_sale_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"City (invoices) with Best Sales:\n {county_inv_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"Category (sales) with Best Sales:\n {catname_sale_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"Category (invoices) with Best Sales:\n {catname_inv_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"Vendor (sales) with Best Sales:\n {vendor_sale_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"Vendor (invoices) with Best Sales:\n {vendor_inv_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"Item (sales) with Best Sales:\n {item_sale_byMonthToMonth_2021}")
print("#-----------------------------------#")
print(f"Item (invoices) with Best Sales:\n {item_inv_byMonthToMonth_2021}")